# PyOR Quantum
## Author: Vineeth Thalakottoor
## Introduction to CSA Hamiltonians

In [1]:
# Define the source path
SourcePath = '/media/HD2/Vineeth/PostDoc_Simulations/Github/PyOR_V1/PyOR_Combined/Source_Doc'

# Add source path
import sys
sys.path.append(SourcePath)

import numpy as np
# Import PyOR package
from PyOR_QuantumSystem import QuantumSystem as QunS
from PyOR_Hamiltonian import Hamiltonian
from PyOR_QuantumLibrary import QuantumLibrary
import PyOR_SphericalTensors as ST
import PyOR_Rotation as Rot
QLib = QuantumLibrary()

In [2]:
# Define the spin system
Spin_list = {"A" : "H1"}
QS = QunS(Spin_list,PrintDefault=False)

# initialize the system
QS.Initialize()

### Set parameters

In [3]:
# Master Equation
QS.PropagationSpace = "Hilbert"
QS.MasterEquation = "Redfield"

# B0 Field in Tesla, Static Magnetic field (B0) along Z
QS.B0 = QS.L100

# Offset Frequency in rotating frame (Hz)
QS.OFFSET["A"] = 0.0


# Define initial and final Spin Temperature
QS.I_spintemp["A"] = 300.0
QS.F_spintemp["A"] = 300.0

# Relaxation Process
QS.Rprocess = "Phenomenological"
QS.R1 = 1
QS.R2 = 2

QS.Update()

Rotating frame frequencies: {'A': -628541601.39}
Offset frequencies: {'A': 0.0}
Initial spin temperatures: {'A': 300.0}
Final spin temperatures: {'A': 300.0}
Radiation damping gain: {'A': 0}
Radiation damping phase: {'A': 0}

Rprocess = Phenomenological
RelaxParDipole_tau = 0.0
DipolePairs = []
RelaxParDipole_bIS = []


### Zeeman Hamiltonians

In [4]:
# generate Larmor Frequencies
QS.print_Larmor = True
Ham = Hamiltonian(QS)

Larmor Frequency in MHz:  [-100.0355028]


In [5]:
# Lab Frame Hamiltonian
Hz_lab = Ham.Zeeman()
Hz_lab.Inverse2PI().matrix

Matrix([
[-50017753.8359235,                0],
[                0, 50017753.8359235]])

In [6]:
# Rotating Frame Hamiltonian
Hz = Ham.Zeeman_RotFrame()
Hz.Inverse2PI().matrix

Matrix([
[0, 0],
[0, 0]])

### CSA tensor PAF

In [7]:
delta_iso = 5.0 # Hz
delta_aniso = -10.0 # Hz

IT_PAF = Ham.InteractionTensor_PAF_CSA(Iso=delta_iso,Aniso=delta_aniso,Asymmetry=1.0)
IT_PAF.Inverse2PI().matrix

Matrix([
[15.0,   0,    0],
[   0, 5.0,    0],
[   0,   0, -5.0]])

In [8]:
PAF_Decom = Ham.InteractionTensor_PAF_Decomposition(IT_PAF)
PAF_Decom

{'Isotropic': 4.999999999999999,
 'Anisotropy': -10.0,
 'Asymmetry': 0.9999999999999999}

### CSA tensor LAB

In [9]:
phi,theta = 20.0,40.0

IT_LAB = Ham.InteractionTensor_LAB(IT_PAF,phi,theta,0)
IT_LAB.Round(3).matrix

Matrix([
[  41.05,   3.507, -58.146],
[  3.507,  32.692, -21.163],
[-58.146, -21.163,  20.505]])

In [10]:
LAB_Decom = Ham.InteractionTensor_LAB_Decomposition(IT_LAB)
LAB_Decom['Isotropic'].matrix

Matrix([
[31.4159265358979,                0,                0],
[               0, 31.4159265358979,                0],
[               0,                0, 31.4159265358979]])

In [11]:
LAB_Decom["Symmetric"].Inverse2PI().Round(3).matrix

Matrix([
[ 6.533,  0.558, -9.254],
[ 0.558,  5.203, -3.368],
[-9.254, -3.368,  3.264]])

In [12]:
LAB_Decom["Antisymmetric"].Inverse2PI().Round(3).matrix

Matrix([
[0, 0, 0],
[0, 0, 0],
[0, 0, 0]])

### CSA Hamiltonian in Lab frame (Secular)

In [13]:
A = "A"
B = ""

Hcsa_LAB_Truncates = Ham.Interaction_Hamiltonian_LAB_CSA_Secular(A,IT_PAF,theta,phi)
Hcsa_LAB_Truncates.Inverse2PI().Round(3).matrix

Matrix([
[1.39,     0],
[   0, -1.39]])

### CSA Hamiltonian in Lab frame (Cartesian, Euler rotation)

In [14]:
Hcsa_LAB_Euler = Ham.Interaction_Hamiltonian_Catesian_Euler(A,IT_PAF,B,0,theta,phi)
Hcsa_LAB_Euler.Inverse2PI().Round(3).matrix

Matrix([
[            1.39, -4.636 + 1.033*I],
[-4.636 - 1.033*I,            -1.39]])

### CSA Hamiltonian in Lab frame (Cartesian, Wigner rotation)

In [15]:
Hcsa_LAB_Wig = Ham.Interaction_Hamiltonian_Catesian_Wigner(A,IT_PAF,B,0,theta,phi)
Hcsa_LAB_Wig.Inverse2PI().Round(3).matrix

Matrix([
[            1.39, -4.636 + 1.033*I],
[-4.636 - 1.033*I,            -1.39]])

### CSA Hamiltonian in Lab frame (Spherical Tensors, Wigner Rotation)

In [16]:
Hcsa_LAB_ST = Ham.Interaction_Hamiltonian_SphericalTensor(A,IT_PAF,B,"spin-field","all", 0,theta,phi)
Hcsa_LAB_ST.Inverse2PI().Round(3).matrix

Matrix([
[            1.39, -4.636 - 1.033*I],
[-4.636 + 1.033*I,            -1.39]])

In [17]:
Hcsa_LAB_ST = Ham.Interaction_Hamiltonian_SphericalTensor(A,IT_PAF,B,"spin-field","secular", 0,theta,phi)
Hcsa_LAB_ST.Inverse2PI().Round(3).matrix

Matrix([
[1.39,     0],
[   0, -1.39]])

In [18]:
Hcsa_LAB_ST = Ham.Interaction_Hamiltonian_SphericalTensor(A,IT_PAF,B,"spin-field","secular + pseudosecular", 0,theta,phi)
Hcsa_LAB_ST.Inverse2PI().Round(3).matrix

Matrix([
[1.39,     0],
[   0, -1.39]])